In [1]:
!git clone https://github.com/2noise/ChatTTS.git
repo_path = "/content/ChatTTS"

Cloning into 'ChatTTS'...
remote: Enumerating objects: 2767, done.
remote: Counting objects: 100% (832/832), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 2767 (delta 587), reused 481 (delta 481), pack-reused 1935 (from 4)
Receiving objects: 100% (2767/2767), 10.42 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (1656/1656), done.


In [2]:
#  INSTALANDO DEPENDENCIAS
!pip install transformers torch accelerate tqdm pandas

In [3]:
#  IMPORTS
import os
from transformers import pipeline
from tqdm import tqdm
import pandas as pd

In [4]:
#  CARREGANDO O MODELo  'bert4re' COMO CLASSIFIcADOR de TEXTO
classifier = pipeline(
    "text-classification",
    model="thearod5/bert4re",
    truncation=True,   # truca se o texto possuir mais de 512 tokens
    max_length=512,    # tamanho max. do input
    top_k=None         # lista de labels
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/708 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at thearod5/bert4re and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

Device set to use cpu


In [5]:
#  LABELS DE CLASSIFICAÇÃO (PADRÕES DE ARQUITETURA)
labels = [
    "pipeline",
    "mvc", "layers", "microservices", "blackboard",
    "event-driven", "client-server", "repository", "pipe-filter"
]

In [6]:
#  DIRETORIO COM O CODIGO FONTE DO CHATTTS (PRECISA CLONAR O REPOSITRIO ANTES)

base_path = "/content/ChatTTS/ChatTTS"

In [7]:
#  FUNCAO PARA COLETAR O CODIGO FONTE
def collect_files(base_path, exts=(".py", ".js", ".ts", ".cpp", ".json", ".md")):
    files = []
    for root, _, fnames in os.walk(base_path):
        for f in fnames:
            if f.endswith(exts):  # only keep source-like files
                files.append(os.path.join(root, f))
    return files

# Collect all matching files in ChatTTS
source_files = collect_files(base_path)
print(f"Found {len(source_files)} source files.")

Found 37 source files.


In [8]:
#  FUNCAO DE CLASSIFICACAO
def classify_file(path, classifier, labels):
    # le o arquivo
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()
    except Exception as e:
        return {"file": path, "error": str(e)}

    # trunca o texto
    if len(content) > 2000:
        content = content[:2000]

    scores = {}  #score para cada label

    # para cada label, classifica o codigo fonte e compara
    for lbl in labels:
        prompt = f"This source code is about {lbl}."
        text = content + "\n" + prompt

        result = classifier(text, truncation=True, max_length=512)

        # normaliza tipo de retorno
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
            r = result[0]
        elif isinstance(result, list) and len(result) > 0 and isinstance(result[0], list):
            r = result[0][0]
        else:
            r = {"label": "LABEL_0", "score": 0.0}

        score = r["score"] if r["label"].startswith("LABEL_1") else 1 - r["score"]
        scores[lbl] = score

    # as 3 labels com o maior score
    top3 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]
    return {"file": path, "top_labels": [lbl for lbl, _ in top3]}

In [9]:
#  EXECUTANDO CLASSIFICACAO NOS ARQUIVOS DO CODIGO FONTE
results = []
for path in tqdm(source_files):
    results.append(classify_file(path, classifier, labels))

#  SALVANDO COMO CSV
df = pd.DataFrame([
    {"file": r["file"], "top_labels": ", ".join(r["top_labels"])}
    for r in results if "top_labels" in r
])
df.to_csv("chattts_architecture_labels.csv", index=False)
print("✅ Results saved to chattts_architecture_labels.csv")

#  PREVIEW
df.head(10)

100%|██████████| 37/37 [03:30<00:00,  5.70s/it]

✅ Results saved to chattts_architecture_labels.csv


,file,top_labels
0,/content/ChatTTS/ChatTTS/norm.py,"pipeline, mvc, layers"
1,/content/ChatTTS/ChatTTS/core.py,"pipeline, mvc, layers"
2,/content/ChatTTS/ChatTTS/__init__.py,"event-driven, pipe-filter, layers"
3,/content/ChatTTS/ChatTTS/model/gpt.py,"pipeline, mvc, layers"
4,/content/ChatTTS/ChatTTS/model/tokenizer.py,"pipeline, mvc, layers"
5,/content/ChatTTS/ChatTTS/model/processors.py,"pipeline, mvc, layers"
6,/content/ChatTTS/ChatTTS/model/__init__.py,"layers, event-driven, pipeline"
7,/content/ChatTTS/ChatTTS/model/dvae.py,"pipeline, mvc, layers"
8,/content/ChatTTS/ChatTTS/model/embed.py,"pipeline, mvc, layers"
9,/content/ChatTTS/ChatTTS/model/speaker.py,"pipeline, mvc, layers"
